In [60]:
import pandas as pd

### 1. Ler base

In [61]:
# Read pickle file
df_censo_pkl = pd.read_pickle('Bases\microdados_ed_basica_2023_tratada.pkl')

In [62]:
df_censo_pkl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178246 entries, 0 to 217624
Data columns (total 69 columns):
 #   Column                           Non-Null Count   Dtype   
---  ------                           --------------   -----   
 0   BIN_AGUA_INEXISTENTE             178246 non-null  bool    
 1   BIN_AGUA_POTAVEL                 178246 non-null  bool    
 2   BIN_AREA_VERDE                   178246 non-null  bool    
 3   BIN_AUDITORIO                    178246 non-null  bool    
 4   BIN_BANHEIRO                     178246 non-null  bool    
 5   BIN_BIBLIOTECA                   178246 non-null  bool    
 6   BIN_ENERGIA_INEXISTENTE          178246 non-null  bool    
 7   BIN_ESGOTO_INEXISTENTE           178246 non-null  bool    
 8   BIN_INTERNET_ALUNOS              178246 non-null  bool    
 9   BIN_INTERNET_APRENDIZAGEM        178246 non-null  bool    
 10  BIN_LABORATORIO_CIENCIAS         178246 non-null  bool    
 11  BIN_LABORATORIO_INFORMATICA      178246 non-null  bool   

### 2. Analisar colunas chave para agrupamento

In [63]:
# value_counts para coluna 'CO_MUNICIPIO'
df_censo_pkl['CHA_CO_MUNICIPIO'].value_counts()

CHA_CO_MUNICIPIO
3550308    7185
3304557    3877
2927408    1532
3106200    1341
2304400    1272
           ... 
2605459       2
3119708       2
3503950       2
5220058       2
5215504       2
Name: count, Length: 5570, dtype: int64

In [64]:
# value_counts para coluna 'TP_DEPENDENCIA'
df_censo_pkl['CHA_DEPENDENCIA'].value_counts()

CHA_DEPENDENCIA
3    106570
4     41463
2     29507
1       706
Name: count, dtype: int64

In [65]:
# value_counts para coluna 'TP_LOCALIZACAO'
df_censo_pkl['CHA_LOCALIZACAO'].value_counts()

CHA_LOCALIZACAO
1    126399
2     51847
Name: count, dtype: int64

In [66]:
# Objetivo: verificar quantos casos diferentes existem agrupando pelas colunas: CO_MUNICIPIO, TP_DEPENDENCIA, TP_LOCALIZACAO
df_censo_pkl.groupby(['CHA_CO_MUNICIPIO', 'CHA_DEPENDENCIA', 'CHA_LOCALIZACAO']).size().reset_index(name='count')

,CHA_CO_MUNICIPIO,CHA_DEPENDENCIA,CHA_LOCALIZACAO,count
0,1100015,2,1,5
1,1100015,2,2,7
2,1100015,3,1,10
3,1100015,3,2,4
4,1100023,1,2,1
...,...,...,...,...
21336,5300108,1,2,2
21337,5300108,2,1,581
21338,5300108,2,2,80
21339,5300108,4,1,588


### 3. Realizar agrupamento pelas três colunas selecionadas

In [67]:
# Identificar colunas por tipo e prefixo
col_bin = [col for col in df_censo_pkl.columns if col.startswith('BIN_') and df_censo_pkl[col].dtype == bool]
col_num = [col for col in df_censo_pkl.columns if col.startswith('NUM_') and pd.api.types.is_integer_dtype(df_censo_pkl[col])]
col_cat = [col for col in df_censo_pkl.columns if col.startswith('CAT_') and df_censo_pkl[col].dtype.name == 'category']

# Dicionário de agregações
agg_dict = {col: 'mean' for col in col_bin}
agg_dict.update({col: 'mean' for col in col_num})
agg_dict.update({col: lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA for col in col_cat})

# Realizar o agrupamento
agrupado = df_censo_pkl.groupby(['CHA_CO_MUNICIPIO', 'CHA_DEPENDENCIA', 'CHA_LOCALIZACAO']).agg(agg_dict).reset_index()

# Preencher NaN com 0 para binárias e numéricas, e pd.NA para categóricas
for col in col_bin + col_num:
    agrupado[col] = agrupado[col].fillna(0)
for col in col_cat:
    agrupado[col] = agrupado[col].fillna(pd.NA)

agrupado.head()

,CHA_CO_MUNICIPIO,CHA_DEPENDENCIA,CHA_LOCALIZACAO,BIN_AGUA_INEXISTENTE,BIN_AGUA_POTAVEL,BIN_AREA_VERDE,BIN_AUDITORIO,BIN_BANHEIRO,BIN_BIBLIOTECA,BIN_ENERGIA_INEXISTENTE,...,NUM_TUR_MED_INT,CAT_CATEGORIA_ESCOLA_PRIVADA,CAT_EXAME_SELECAO,CAT_LOCALIZACAO_DIFERENCIADA,CAT_OCUPACAO_GALPAO,CAT_OCUPACAO_PREDIO_ESCOLAR,CAT_ORGAO_REGIONAL,CAT_PROPOSTA_PEDAGOGICA,CAT_REGIAO,CAT_TRATAMENTO_LIXO_INEXISTENTE
0,1100015,2,1,0.0,1.0,0.4,0.4,1.000000,1.0,0.000000,...,1.6,0.0,0.0,0.0,0.0,1.0,00015,1.0,1,0.0
1,1100015,2,2,0.0,1.0,0.0,0.0,0.714286,0.0,0.714286,...,0.0,0.0,0.0,2.0,0.0,1.0,00015,1.0,1,1.0
2,1100015,3,1,0.0,1.0,0.1,0.0,1.000000,0.2,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,00015,1.0,1,1.0
3,1100015,3,2,0.0,1.0,0.0,0.0,1.000000,0.5,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,00015,1.0,1,1.0
4,1100023,1,2,0.0,1.0,1.0,1.0,1.000000,1.0,0.000000,...,18.0,0.0,1.0,0.0,0.0,1.0,0,1.0,1,1.0


In [68]:
# Agrupadas
print(len(col_bin), "colunas binárias")
print(len(col_num), "colunas numéricas")
print(len(col_cat), "colunas categóricas")

40 colunas binárias
17 colunas numéricas
9 colunas categóricas


In [69]:
agrupado

,CHA_CO_MUNICIPIO,CHA_DEPENDENCIA,CHA_LOCALIZACAO,BIN_AGUA_INEXISTENTE,BIN_AGUA_POTAVEL,BIN_AREA_VERDE,BIN_AUDITORIO,BIN_BANHEIRO,BIN_BIBLIOTECA,BIN_ENERGIA_INEXISTENTE,...,NUM_TUR_MED_INT,CAT_CATEGORIA_ESCOLA_PRIVADA,CAT_EXAME_SELECAO,CAT_LOCALIZACAO_DIFERENCIADA,CAT_OCUPACAO_GALPAO,CAT_OCUPACAO_PREDIO_ESCOLAR,CAT_ORGAO_REGIONAL,CAT_PROPOSTA_PEDAGOGICA,CAT_REGIAO,CAT_TRATAMENTO_LIXO_INEXISTENTE
0,1100015,2,1,0.0,1.0,0.400000,0.400000,1.000000,1.000000,0.000000,...,1.600000,0.0,0.0,0.0,0.0,1.0,00015,1.0,1,0.0
1,1100015,2,2,0.0,1.0,0.000000,0.000000,0.714286,0.000000,0.714286,...,0.000000,0.0,0.0,2.0,0.0,1.0,00015,1.0,1,1.0
2,1100015,3,1,0.0,1.0,0.100000,0.000000,1.000000,0.200000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,1.0,00015,1.0,1,1.0
3,1100015,3,2,0.0,1.0,0.000000,0.000000,1.000000,0.500000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,1.0,00015,1.0,1,1.0
4,1100023,1,2,0.0,1.0,1.000000,1.000000,1.000000,1.000000,0.000000,...,18.000000,0.0,1.0,0.0,0.0,1.0,0,1.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21336,5300108,1,2,0.0,1.0,1.000000,1.000000,1.000000,1.000000,0.000000,...,4.500000,0.0,0.0,0.0,0.0,1.0,0,1.0,5,0.0
21337,5300108,2,1,0.0,1.0,0.614458,0.165232,1.000000,0.270224,0.000000,...,0.115318,0.0,0.0,0.0,0.0,1.0,00001,1.0,5,0.0
21338,5300108,2,2,0.0,1.0,0.775000,0.025000,1.000000,0.187500,0.000000,...,0.000000,0.0,0.0,0.0,0.0,1.0,00006,1.0,5,1.0
21339,5300108,4,1,0.0,1.0,0.556122,0.319728,1.000000,0.464286,0.000000,...,0.088435,1.0,0.0,0.0,0.0,1.0,00099,1.0,5,0.0


### 4. Ajustar tipos do agrupamento

In [70]:
# Colunas binárias foi aplicado o método mean, então os valores são 0 ou 1
# Colunas numéricas foi aplicado o método mean, então os valores são médias
# Colunas categóricas foi aplicado o método mode, então os valores são os mais frequentes

# Colunas com o prefixo 'BIN_' e agrupadas com mean variando entre 0 e 1 -> tipo deve ser ajustado para float
for col in col_bin:
    agrupado[col] = agrupado[col].astype(float)

# Colunas com o prefixo 'NUM_' e agrupadas com mean -> tipo deve ser ajustado para float
for col in col_num:
    agrupado[col] = agrupado[col].astype(float)

# Colunas com o prefixo 'CAT_' e agrupadas com mode -> tipo deve ser ajustado para category
for col in col_cat:
    agrupado[col] = agrupado[col].astype('category')

In [71]:
# Renomear colunas para prefixos representarem os novos tipos
agrupado.rename(columns=lambda x: x.replace('NUM_', 'NUM_MEAN_').replace('BIN_', 'NUM_PERC_').replace('CAT_', 'CAT_MODE_'), inplace=True)

In [72]:
# Ordernar colunas por ordem alfabética
agrupado = agrupado.reindex(sorted(agrupado.columns), axis=1)

In [73]:
agrupado

,CAT_MODE_CATEGORIA_ESCOLA_PRIVADA,CAT_MODE_EXAME_SELECAO,CAT_MODE_LOCALIZACAO_DIFERENCIADA,CAT_MODE_OCUPACAO_GALPAO,CAT_MODE_OCUPACAO_PREDIO_ESCOLAR,CAT_MODE_ORGAO_REGIONAL,CAT_MODE_PROPOSTA_PEDAGOGICA,CAT_MODE_REGIAO,CAT_MODE_TRATAMENTO_LIXO_INEXISTENTE,CHA_CO_MUNICIPIO,...,NUM_PERC_PISCINA,NUM_PERC_PODER_PUBLICO_PARCERIA,NUM_PERC_PROF_TEC,NUM_PERC_QUADRA_ESPORTES,NUM_PERC_SALA_ATELIE_ARTES,NUM_PERC_SALA_MUSICA_CORAL,NUM_PERC_VINCULO_OUTRO_ORGAO,NUM_PERC_VINCULO_SECRETARIA_EDUCACAO,NUM_PERC_VINCULO_SECRETARIA_SAUDE,NUM_PERC_VINCULO_SEGURANCA_PUBLICA
0,0.0,0.0,0.0,0.0,1.0,00015,1.0,1,0.0,1100015,...,0.000000,0.000000,0.000000,1.000000,0.200000,0.200000,0.0,0.800000,0.000000,0.200000
1,0.0,0.0,2.0,0.0,1.0,00015,1.0,1,1.0,1100015,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,1.0,00015,1.0,1,1.0,1100015,...,0.000000,0.000000,0.000000,0.600000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000
3,0.0,0.0,0.0,0.0,1.0,00015,1.0,1,1.0,1100015,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000
4,0.0,1.0,0.0,0.0,1.0,0,1.0,1,1.0,1100023,...,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21336,0.0,0.0,0.0,0.0,1.0,0,1.0,5,0.0,5300108,...,0.000000,0.000000,1.000000,1.000000,0.000000,0.500000,0.0,1.000000,0.000000,0.000000
21337,0.0,0.0,0.0,0.0,1.0,00001,1.0,5,0.0,5300108,...,0.044750,0.000000,0.022375,0.740103,0.027539,0.020654,0.0,0.994836,0.001721,0.024096
21338,0.0,0.0,0.0,0.0,1.0,00006,1.0,5,1.0,5300108,...,0.025000,0.000000,0.000000,0.675000,0.012500,0.025000,0.0,1.000000,0.000000,0.000000
21339,1.0,0.0,0.0,0.0,1.0,00099,1.0,5,0.0,5300108,...,0.166667,0.013605,0.074830,0.557823,0.161565,0.127551,0.0,0.000000,0.000000,0.000000


In [74]:
agrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21341 entries, 0 to 21340
Data columns (total 69 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   CAT_MODE_CATEGORIA_ESCOLA_PRIVADA     21341 non-null  category
 1   CAT_MODE_EXAME_SELECAO                21341 non-null  category
 2   CAT_MODE_LOCALIZACAO_DIFERENCIADA     21341 non-null  category
 3   CAT_MODE_OCUPACAO_GALPAO              21341 non-null  category
 4   CAT_MODE_OCUPACAO_PREDIO_ESCOLAR      21341 non-null  category
 5   CAT_MODE_ORGAO_REGIONAL               21341 non-null  category
 6   CAT_MODE_PROPOSTA_PEDAGOGICA          21341 non-null  category
 7   CAT_MODE_REGIAO                       21341 non-null  category
 8   CAT_MODE_TRATAMENTO_LIXO_INEXISTENTE  21341 non-null  category
 9   CHA_CO_MUNICIPIO                      21341 non-null  int64   
 10  CHA_DEPENDENCIA                       21341 non-null  int64   
 11  CH

### 5. Salvar base agrupada

In [75]:
# Salvar o DataFrame agrupado em um novo arquivo pickle
agrupado.to_pickle('Bases\microdados_ed_basica_2023_agrupado.pkl')